<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="45%" align="right" border="4">

# Framework Classes and Functions

This section explains the usage of some basic framework classes and functions of DX Analytics. Mainly some helper functions, the discounting classes and the market environment class used to store market data and other parameters/data needed to model, value and risk manage derivative instruments. 

In [1]:
import warnings; warnings.simplefilter('ignore')

In [2]:
from dx import *

In [3]:
np.set_printoptions(precision=3)

## Helper Functions

There are two helper functions used regulary:

* <code>get_year_deltas</code>: get a list of year deltas (decimal fractions) relative to first value in time_list
* <code>sn_random_numbers</code>: get an array of standard normally distributed pseudo-random numbers

### get_year_deltas

Suppose we have a `list` object containing a number of `datetime` objects.

In [4]:
time_list = [dt.datetime(2015, 1, 1),
             dt.datetime(2015, 4, 1),
             dt.datetime(2015, 6, 15),
             dt.datetime(2015, 10, 21)]

Passing this object to the `get_year_deltas` functions yields a list of year fractions representing the time intervals between the dates given. This is sometimes used e.g. for discounting purposes.

In [5]:
get_year_deltas(time_list)

array([ 0.   ,  0.247,  0.452,  0.803])

### sn_random_numbers

Monte Carlo simulation of course relies heavily an the use of random numbers. The function `sn_random_numbers` is a wrapper function around the pseudo-random number generator of the `NumPy` library. It implements antithetic variates and moment matching as generic variance reduction techniques. It also allows to fix the seed value for the random number generator. The `shape` parameter is a `tuple` object of three integers.

In [6]:
ran = sn_random_numbers((2, 3, 4), antithetic=True,
                        moment_matching=True, fixed_seed=False)

In [7]:
ran

array([[[ 1.676,  0.929, -1.676, -0.929],
        [ 0.169, -1.436, -0.169,  1.436],
        [ 1.826, -1.04 , -1.826,  1.04 ]],

       [[-1.108,  0.214,  1.108, -0.214],
        [-0.261, -0.467,  0.261,  0.467],
        [ 0.412, -0.299, -0.412,  0.299]]])

Using moment matching makes sure that the first and second moments match exactly 0 and 1, respectively.

In [8]:
ran.mean()

0.0

In [9]:
ran.std()

1.0

Setting the first value of the `shape` parameter to 1 yields a two-dimensional `ndarray` object.

In [10]:
ran = sn_random_numbers((1, 3, 4), antithetic=True,
                        moment_matching=True, fixed_seed=False)

In [11]:
ran

array([[-0.849, -0.195,  0.849,  0.195],
       [ 1.685,  0.158, -1.685, -0.158],
       [-1.092, -1.088,  1.092,  1.088]])

## Discounting Classes

In the risk-neutral valuation of derivative instrumente, discounting payoffs is a major task. The following discounting classes are implemented:

* `constant_short_rate`: fixed short rate
* `deterministic_yield`: deterministic yiels/term structure

### constant_short_rate

The `constant_short_rate` class represents the most simple case for risk-neutral discounting. A discounting object is defined by instatiating the class and providing a name and a decimal short rate value only.

In [12]:
r = constant_short_rate('r', 0.05)

In [13]:
r.name

'r'

In [14]:
r.short_rate

0.05

The object has a method `get_forward_rates` to generate forward rates given, for instance, a `list` object of `datetime` objects.

In [15]:
r.get_forward_rates(time_list)

([datetime.datetime(2015, 1, 1, 0, 0),
  datetime.datetime(2015, 4, 1, 0, 0),
  datetime.datetime(2015, 6, 15, 0, 0),
  datetime.datetime(2015, 10, 21, 0, 0)],
 array([ 0.05,  0.05,  0.05,  0.05]))

Similarly, the method `get_discount_factors` returns discount factors for such a `list` object.

In [16]:
r.get_discount_factors(time_list)

([datetime.datetime(2015, 1, 1, 0, 0),
  datetime.datetime(2015, 4, 1, 0, 0),
  datetime.datetime(2015, 6, 15, 0, 0),
  datetime.datetime(2015, 10, 21, 0, 0)],
 array([ 0.961,  0.978,  0.988,  1.   ]))

You can also pass, for instance, an `ndarry` object containing year fractions.

In [17]:
r.get_discount_factors(np.array([0., 1., 1.5, 2.]),
                       dtobjects=False)

(array([ 0. ,  1. ,  1.5,  2. ]), array([ 0.905,  0.928,  0.951,  1.   ]))

### deterministic_short_rate

The `deterministic_short_rate` class allows to model an interest rate term structure. To this end, you need to pass a `list` object of `datetime` and yield pairs to the  class.

In [18]:
yields = [(dt.datetime(2015, 1, 1), 0.02),
          (dt.datetime(2015, 3, 1), 0.03),
          (dt.datetime(2015, 10, 15), 0.035),
          (dt.datetime(2015, 12, 31), 0.04)]

In [19]:
y = deterministic_short_rate('y', yields)

In [20]:
y.name

'y'

In [21]:
y.yield_list

array([[datetime.datetime(2015, 1, 1, 0, 0), 0.02],
       [datetime.datetime(2015, 3, 1, 0, 0), 0.03],
       [datetime.datetime(2015, 10, 15, 0, 0), 0.035],
       [datetime.datetime(2015, 12, 31, 0, 0), 0.04]], dtype=object)

The method `get_interpolated_yields` implements an interpolation of the yield data and returns the interpolated yields given a `list` object of `datetime` objects.

In [22]:
y.get_interpolated_yields(time_list)

array([[datetime.datetime(2015, 1, 1, 0, 0), 0.019999999999999993,
        0.0840608597791612],
       [datetime.datetime(2015, 4, 1, 0, 0), 0.03283048934520345,
        0.02532998361805569],
       [datetime.datetime(2015, 6, 15, 0, 0), 0.035133049718591186,
        0.0007769642303797064],
       [datetime.datetime(2015, 10, 21, 0, 0), 0.03515012570984609,
        0.010083939037494674]], dtype=object)

In similar fashion, the methods `get_forward_rates` and `get_discount_factors` return forward rates and discount factors, respcectively.

In [23]:
y.get_forward_rates(time_list)

([datetime.datetime(2015, 1, 1, 0, 0),
  datetime.datetime(2015, 4, 1, 0, 0),
  datetime.datetime(2015, 6, 15, 0, 0),
  datetime.datetime(2015, 10, 21, 0, 0)],
 array([0.019999999999999993, 0.039076238730477456, 0.0354842801241053,
        0.04324490417008154], dtype=object))

In [24]:
y.get_discount_factors(time_list)

([datetime.datetime(2015, 1, 1, 0, 0),
  datetime.datetime(2015, 4, 1, 0, 0),
  datetime.datetime(2015, 6, 15, 0, 0),
  datetime.datetime(2015, 10, 21, 0, 0)],
 [0.97166103139227611, 0.97876383482361962, 0.98629027682763593, 1.0])

## Market Environment

The `market_environment` class is used to collect relevant data for the modeling, valuation and risk management of single derivatives instruments and portfolios composed of such instruments. A `market_environment` object stores:

* `constants`: e.g. maturity date of option
* `lists`: e.g. list of dates
* `curves`: e.g. discounting objects

A `market_environment` object is instantiated by providing a name as a `string` object and the pricing date as a `datetime` object.

In [25]:
me = market_environment(name='me', pricing_date=dt.datetime(2014, 1, 1))

Constants are added via the `add_constant` method and providing a key and the value.

In [26]:
me.add_constant('initial_value', 100.)

In [27]:
me.add_constant('volatility', 0.25)

Lists of data are added via the  `add_list` method.

In [28]:
me.add_list('dates', time_list)

The `add_curve` method does the same for curves.

In [29]:
me.add_curve('discount_curve_1', r)

In [30]:
me.add_curve('discount_curve_2', y)

The single data objects are stored in separate `dictionary` objects.

In [31]:
me.constants

{'initial_value': 100.0, 'volatility': 0.25}

In [32]:
me.lists

{'dates': [datetime.datetime(2015, 1, 1, 0, 0),
  datetime.datetime(2015, 4, 1, 0, 0),
  datetime.datetime(2015, 6, 15, 0, 0),
  datetime.datetime(2015, 10, 21, 0, 0)]}

In [33]:
me.curves

{'discount_curve_1': <dx.dx_frame.constant_short_rate at 0x7fca8b8985d0>,
 'discount_curve_2': <dx.dx_frame.deterministic_short_rate at 0x7fca8b898f90>}

Data is retrieved from a `market_environment` object via the `get_constant`, `get_list` and `get_curve` methods and providing the respective key.

In [34]:
me.get_constant('volatility')

0.25

In [35]:
me.get_list('dates')

[datetime.datetime(2015, 1, 1, 0, 0),
 datetime.datetime(2015, 4, 1, 0, 0),
 datetime.datetime(2015, 6, 15, 0, 0),
 datetime.datetime(2015, 10, 21, 0, 0)]

In [36]:
me.get_curve('discount_curve_1')

Retrieving, for instance, a discounting object you can in one step retrieve it and call a method on it.

In [37]:
me.get_curve('discount_curve_2').get_discount_factors(time_list)

([datetime.datetime(2015, 1, 1, 0, 0),
  datetime.datetime(2015, 4, 1, 0, 0),
  datetime.datetime(2015, 6, 15, 0, 0),
  datetime.datetime(2015, 10, 21, 0, 0)],
 [0.97166103139227611, 0.97876383482361962, 0.98629027682763593, 1.0])

**Copyright, License & Disclaimer**

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

DX Analytics (the "dx library") is licensed under the GNU Affero General Public License
version 3 or later (see [http://www.gnu.org/licenses/](http://www.gnu.org/licenses/)).

DX Analytics comes with no representations
or warranties, to the extent permitted by applicable law.


<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

[http://tpq.io](http://tpq.io) | [team@tpq.io](mailto:team@tpq.io) | [http://twitter.com/dyjh](http://twitter.com/dyjh)

**Quant Platform** |
[http://quant-platform.com](http://quant-platform.com)

**Derivatives Analytics with Python (Wiley Finance)** |
[http://derivatives-analytics-with-python.com](http://derivatives-analytics-with-python.com)

**Python for Finance (O'Reilly)** |
[http://python-for-finance.com](http://python-for-finance.com)